In [1]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
import shutil

shutil.rmtree("/app/_data/ScaledYOLOv4/mish-cuda/build/")
!pip install /app/_data/ScaledYOLOv4/mish-cuda

Processing /app/_data/ScaledYOLOv4/mish-cuda
  Preparing metadata (setup.py) ... done
  Created wheel for mish-cuda: filename=mish_cuda-0.0.3-cp38-cp38-linux_x86_64.whl size=2973503 sha256=69d4b110b4d7d5b3a80ae9ee8efa8a5885470224057ec0447ee5882eccba3309
  Stored in directory: /root/.cache/pip/wheels/c8/2b/04/ed21c9eff9a11a2ee9114399ca6249ac2e31c5833196e1aa82
Successfully built mish-cuda
  Attempting uninstall: mish-cuda
    Found existing installation: mish-cuda 0.0.3
    Uninstalling mish-cuda-0.0.3:
      Successfully uninstalled mish-cuda-0.0.3
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [3]:
import mish_cuda

In [4]:
import ast
import glob
import json
import os
import re
import shutil

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from IPython.core.interactiveshell import InteractiveShell
from PIL import Image
from sklearn.model_selection import GroupKFold, KFold
from tqdm import tqdm

InteractiveShell.ast_node_interactivity = "all"
import seaborn as sns
import torch
import torchvision
from IPython.display import clear_output

In [5]:
TRAIN_DF_PART = "/app/_data/tensorflow-great-barrier-reef/train.csv"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE
SEED = 42
IMAGE_FOLDER = "images"

LABEL_FOLDER = IMAGE_FOLDER.replace("images", "labels")
LABEL_FOLDER

'cuda'

'labels'

In [6]:
df = pd.read_csv(TRAIN_DF_PART)
df["img_path"] = (
    "/app/_data/tensorflow-great-barrier-reef/train_images/video_"
    + df.video_id.astype("str")
    + "/"
    + df.video_frame.astype("str")
    + ".jpg"
)
df["annotations"] = df["annotations"].apply(lambda x: ast.literal_eval(x))
df["len_annotation"] = df["annotations"].str.len()
df["image_id"] = df["image_id"].str.replace("-", "_", regex=True)
df["new_img_path"] = f"/app/_data/{IMAGE_FOLDER}/" + df["image_id"] + ".jpg"
train_df = pd.concat(
    [
        df.query("len_annotation!=0"),
        df.query("len_annotation==0").sample(
            6000 - len(df.query("len_annotation!=0")), random_state=SEED
        ),
    ],
    ignore_index=True,
)
train_df = train_df.sort_values(["video_id", "video_frame"]).reset_index(drop=True)

In [7]:
gr_init = 0
count = 0

for i in train_df.index.tolist():
    if i != len(train_df) - 1:
        if train_df.loc[i, "video_frame"] + 1 == train_df.loc[i + 1, "video_frame"]:
            train_df.loc[i, "group"] = gr_init
            train_df.loc[i + 1, "group"] = gr_init
        else:
            train_df.loc[i, "group"] = gr_init
            gr_init += 1
    else:
        if train_df.loc[i, "video_frame"] == train_df.loc[i - 1, "video_frame"] + 1:
            train_df.loc[i, "group"] = gr_init - 1
        else:
            train_df.loc[i, "group"] = gr_init
train_df["group"] = train_df["group"].astype("int")

# Split on video_id

In [8]:
VIDEO_ID = 2
train  = pd.concat(
    [
        df.query("video_id!=@VIDEO_ID and len_annotation!=0"),
        df.query("video_id!=@VIDEO_ID and len_annotation==0").sample(
            int(
                df.query("video_id!=@VIDEO_ID and len_annotation!=0").shape[0]
                * 0.2
            )
        ),
    ]
).sample(frac = 1).reset_index(drop=True)
val = df.query("video_id==@VIDEO_ID").sample(frac = 1).reset_index(drop=True)

In [9]:
train_txt = f"/app/_data/y4_train_val{VIDEO_ID}.txt"
val_txt = train_txt.replace("train_", "val_")
train_txt, val_txt

('/app/_data/y4_train_val2.txt', '/app/_data/y4_val_val2.txt')

In [10]:
train_img_path = train["new_img_path"].tolist()
val_img_path = val["new_img_path"].tolist()
np.savetxt(
    train_txt,
    train_img_path,
    fmt="%s",
)
np.savetxt(val_txt, val_img_path, fmt="%s")

## Custimize parameters

In [15]:
# customize iPython writefile so we can write variables

from IPython.core.magic import register_line_cell_magic


@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, "w") as f:
        f.write(cell.format(**globals()))

In [16]:
%%writetemplate /app/_data/ScaledYOLOv4/data/my_data.yaml

train: {train_txt} # training directory
val: {val_txt} # validation directory

nc: 1 # number of classes
names: ['starfish'] # name of the class

In [17]:
!cat /app/_data/ScaledYOLOv4/data/my_data.yaml


train: /app/_data/y4_train_val2.txt # training directory
val: /app/_data/y4_val_val2.txt # validation directory

nc: 1 # number of classes
names: ['starfish'] # name of the class


In [18]:
%%writetemplate /app/_data/ScaledYOLOv4/data/CustomYolov4p7.yaml
# parameters
nc: 1  # number of classes
depth_multiple: 1.0  # expand model depth
width_multiple: 1.25  # expand layer channels

# anchors
anchors:
  - [13,17,  22,25,  27,66,  55,41]  # P3/8
  - [57,88,  112,69,  69,177,  136,138]  # P4/16
  - [136,138,  287,114,  134,275,  268,248]  # P5/32
  - [268,248,  232,504,  445,416,  640,640]  # P6/64
  - [812,393,  477,808,  1070,908,  1408,1408]  # P7/128

# csp-p7 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [32, 3, 1]],  # 0
   [-1, 1, Conv, [64, 3, 2]],  # 1-P1/2
   [-1, 1, BottleneckCSP, [64]],
   [-1, 1, Conv, [128, 3, 2]],  # 3-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 5-P3/8
   [-1, 15, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 7-P4/16
   [-1, 15, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]], # 9-P5/32
   [-1, 7, BottleneckCSP, [1024]],
   [-1, 1, Conv, [1024, 3, 2]], # 11-P6/64
   [-1, 7, BottleneckCSP, [1024]],
   [-1, 1, Conv, [1024, 3, 2]], # 13-P7/128
   [-1, 7, BottleneckCSP, [1024]],  # 14
  ]

# yolov4-p7 head
# na = len(anchors[0])
head:
  [[-1, 1, SPPCSP, [512]], # 15
   [-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [-6, 1, Conv, [512, 1, 1]], # route backbone P6
   [[-1, -2], 1, Concat, [1]],
   [-1, 3, BottleneckCSP2, [512]], # 20 
   [-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [-13, 1, Conv, [512, 1, 1]], # route backbone P5
   [[-1, -2], 1, Concat, [1]],
   [-1, 3, BottleneckCSP2, [512]], # 25
   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [-20, 1, Conv, [256, 1, 1]], # route backbone P4
   [[-1, -2], 1, Concat, [1]],
   [-1, 3, BottleneckCSP2, [256]], # 30
   [-1, 1, Conv, [128, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [-27, 1, Conv, [128, 1, 1]], # route backbone P3
   [[-1, -2], 1, Concat, [1]],
   [-1, 3, BottleneckCSP2, [128]], # 35
   [-1, 1, Conv, [256, 3, 1]],
   [-2, 1, Conv, [256, 3, 2]],
   [[-1, 30], 1, Concat, [1]],  # cat
   [-1, 3, BottleneckCSP2, [256]], # 39
   [-1, 1, Conv, [512, 3, 1]],
   [-2, 1, Conv, [512, 3, 2]],
   [[-1, 25], 1, Concat, [1]],  # cat
   [-1, 3, BottleneckCSP2, [512]], # 43
   [-1, 1, Conv, [1024, 3, 1]],
   [-2, 1, Conv, [512, 3, 2]],
   [[-1, 20], 1, Concat, [1]],  # cat
   [-1, 3, BottleneckCSP2, [512]], # 47
   [-1, 1, Conv, [1024, 3, 1]],
   [-2, 1, Conv, [512, 3, 2]],
   [[-1, 15], 1, Concat, [1]],  # cat
   [-1, 3, BottleneckCSP2, [512]], # 51
   [-1, 1, Conv, [1024, 3, 1]],

   [[36,40,44,48,52], 1, Detect, [nc, anchors]],   # Detect(P3, P4, P5, P6, P7)
  ]

In [19]:
!cat /app/_data/ScaledYOLOv4/data/hyp.scratch.yaml

# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials


lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
giou: 0.05  # GIoU loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.5  # image translation (+/- fraction)
scale: 0.

In [20]:
%%writetemplate /app/_data/ScaledYOLOv4/data/CustomHyp.yaml

# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials


lr0: 0.007  # initial learning rate (SGD=1E-2, Adam=1E-3)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
giou: 0.05  # GIoU loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.3  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.5  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.5  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mixup: 0.3  # image mixup (probability)


# Train


In [22]:
NAME = f"yolov4p7_val{VIDEO_ID}"
NAME

'yolov4p7_val2'

In [28]:
%cd /app/_data/ScaledYOLOv4/
!git checkout yolov4-large
!python train.py \
        --batch-size 1 \
        --data /app/_data/ScaledYOLOv4/data/my_data.yaml \
        --cfg /app/_data/ScaledYOLOv4/data/CustomYolov4p7.yaml \
        --weights /app/_data/ScaledYOLOv4/models/weights/yolov4-p7.pt \
        --name {NAME} \
        --epochs 50 \
        --hyp /app/_data/ScaledYOLOv4/data/CustomHyp.yaml \
        --img-size 2560 \
        --single-cls

/app/_data/ScaledYOLOv4
M	detect.py
M	models/yolo.py
M	test.py
M	train.py
M	utils/general.py
Already on 'yolov4-large'
Your branch is up to date with 'origin/yolov4-large'.
Using CUDA device0 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3090', total_memory=24265MB)

Namespace(adam=False, batch_size=1, bucket='', cache_images=False, cfg='/app/_data/ScaledYOLOv4/data/CustomYolov4p7.yaml', data='/app/_data/ScaledYOLOv4/data/my_data.yaml', device='', epochs=50, evolve=False, global_rank=-1, hyp='/app/_data/ScaledYOLOv4/data/CustomHyp.yaml', img_size=[2560, 2560], local_rank=-1, logdir='runs/', multi_scale=False, name='yolov4p7_val2', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, single_cls=True, sync_bn=False, total_batch_size=1, weights='/app/_data/ScaledYOLOv4/models/weights/yolov4-p7.pt', world_size=1)
Start Tensorboard with "tensorboard --logdir runs/", view at http://localhost:9091/
Hyperparameters {'lr0': 0.007, 'momentum': 0.937, 'weight_decay': 0.0005,